In [1]:
#danaderp May6'19
#Prediction For Main Issues Data Set

In [2]:
import csv
from tensorflow.keras.preprocessing import text
from nltk.corpus import gutenberg
from string import punctuation
from tensorflow.keras.preprocessing.sequence import skipgrams

/scratch/danaderp/.conda/envs/drmccr_conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Limited tf.compat.v2.summary API due to missing TensorBoard installation
Limited tf.summary API due to missing TensorBoard installation


In [3]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
pd.options.display.max_colwidth = 200
%matplotlib inline

In [4]:
from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dot, Input, Dense, Reshape, LSTM, Conv2D, Flatten, MaxPooling1D, Dropout, MaxPooling2D
from tensorflow.keras.layers import Embedding, Multiply, Subtract
from tensorflow.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

In [6]:
# visualize model structure
#from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.manifold import TSNE

In [7]:
from datasets.read_data import Dynamic_Dataset,Processing_Dataset
from vectorize_sentence import Embeddings

In [8]:
path = "datasets/combined_dataset/"
process_unit = Processing_Dataset(path)
ground_truth = process_unit.get_ground_truth()

In [9]:
dataset = Dynamic_Dataset(ground_truth, path)

In [10]:
test, train = process_unit.get_test_and_training(ground_truth)

In [11]:
print(len(test))
print(len(train))
print(test[0])
print(train[0])

1030
9308
('(1,0)', 'Moved SecurityUserValueResolver to security-http\n\n\nThe SecurityUserValueResolver class is deprecated and will be removed in 5.0, use Symfony\\Component\\Security\\Http\\Controller\\UserValueResolver instead.\nsymfony/symfony#25588')
('(1,0)', "'This is a useful security improvement, that I recommend gets integrated into gitlab. It protects users, in the event that their passwords get stolen from other sites, etc. I found a good gem for this: http://rubydoc.info/github/mdp/rotp/master/frames, however, given that it appears Gitlab uses Devise for auth, we should probably use this plugin: https://github.com/wmlele/devise-otp\n\nI intend to submit a Merge Request for this, so I'll outline my design for the system here (in case anyone has feedback/wants to help):\n\n### OTP Strategy\nI'm going with time-based (TOTP). Its requires no storage implications, per-user (other than a 32bit secret key). Time-based keys are very common, Google uses this strategy to protect GM

In [13]:
#Train/Test split verification
#for elem in test:
#    print(elem[0])

In [14]:
#Preprocesing Corpora
embeddings = Embeddings()
pre_corpora_train = [embeddings.preprocess(doc[1]) for doc in train]
pre_corpora_test = [embeddings.preprocess(doc[1]) for doc in test]

In [15]:
pre_corpora_train[0]

['use',
 'secur',
 'improv',
 'recommend',
 'get',
 'integr',
 'gitlab',
 'protect',
 'user',
 'event',
 'password',
 'get',
 'stolen',
 'site',
 'etc',
 'found',
 'good',
 'gem',
 'http',
 'rubydoc',
 'info',
 'github',
 'mdp',
 'rotp',
 'master',
 'frame',
 'howev',
 'given',
 'appear',
 'gitlab',
 'use',
 'devis',
 'auth',
 'probabl',
 'use',
 'plugin',
 'https',
 'github',
 'com',
 'wmlele',
 'devis',
 'otp',
 'intend',
 'submit',
 'merg',
 'request',
 'outlin',
 'design',
 'system',
 'case',
 'anyon',
 'feedback',
 'want',
 'help',
 'otp',
 'strategi',
 'time',
 'base',
 'totp',
 'requir',
 'storag',
 'implic',
 'per',
 'user',
 'bit',
 'secret',
 'key',
 'time',
 'base',
 'key',
 'common',
 'googl',
 'use',
 'strategi',
 'protect',
 'gmail',
 'app',
 'custom',
 'databas',
 'augment',
 'note',
 'given',
 'exist',
 'devis',
 'otp',
 'may',
 'longer',
 'necessari',
 'add',
 'new',
 'tabl',
 'foreign',
 'key',
 'refer',
 'user',
 'column',
 'totp',
 'secret',
 'column',
 'exist',
 'r

In [16]:
max_len_sentences_train = max([len(doc) for doc in pre_corpora_train]) #<------- [Parameter]
max_len_sentences_test = max([len(doc) for doc in pre_corpora_test]) #<------- [Parameter]

In [17]:
max_len_sentences = max(max_len_sentences_train,max_len_sentences_test)
print("Max. Sentence # words:",max_len_sentences)

Max. Sentence # words: 13806


In [18]:
embed_path = 'datasets/word_embeddings-embed_size_100-epochs_100.csv'
embeddings_dict = embeddings.get_embeddings_dict(embed_path)

In [19]:
corpora_train = [embeddings.vectorize(doc[1], embeddings_dict) for doc in train]#vectorization Inputs
corpora_test = [embeddings.vectorize(doc[1], embeddings_dict) for doc in test]#vectorization

In [20]:
target_train = [[int(list(doc[0])[1]),int(list(doc[0])[3])] for doc in train]#vectorization Output
target_test = [[int(list(doc[0])[1]),int(list(doc[0])[3])]for doc in test]#vectorization Output
#target_train

In [21]:
max_len_sentences_train = max([len(doc) for doc in corpora_train]) #<------- [Parameter]
max_len_sentences_test = max([len(doc) for doc in corpora_test]) #<------- [Parameter]

In [22]:
max_len_sentences = max(max_len_sentences_train,max_len_sentences_test)
print("Max. Sentence # words:",max_len_sentences)

Max. Sentence # words: 13789


In [31]:
min_len_sentences_train = min([len(doc) for doc in corpora_train]) #<------- [Parameter]
min_len_sentences_test = min([len(doc) for doc in corpora_test]) #<------- [Parameter]

In [32]:
min_len_sentences = max(min_len_sentences_train,min_len_sentences_test)
print("Mix. Sentence # words:",min_len_sentences)

Mix. Sentence # words: 1


In [23]:
embed_size = np.size(corpora_train[0][0])

In [24]:
#BaseLine Architecture <-------
embeddigs_cols = embed_size
input_sh = (max_len_sentences,embeddigs_cols,1)
#Selecting filters? 
#https://stackoverflow.com/questions/48243360/how-to-determine-the-filter-parameter-in-the-keras-conv2d-function
#https://stats.stackexchange.com/questions/196646/what-is-the-significance-of-the-number-of-convolution-filters-in-a-convolutional

N_filters = 128 # <-------- [HyperParameter] Powers of 2 Numer of Features
K = 2 # <-------- [HyperParameter] Number of Classess

In [25]:
input_sh

(13789, 100, 1)

In [38]:
#baseline_model = Sequential()
gram_input = Input(shape = input_sh)

In [50]:
# 1st Convolutional Layer
conv_1_layer = Conv2D(filters=64, input_shape=input_sh, activation='relu', 
                      kernel_size=(7,7), strides=(2,2), padding='valid')(gram_input)

In [51]:
conv_1_layer.shape

TensorShape([None, 6892, 47, 64])

In [52]:
# Max Pooling
max_1_pooling = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(conv_1_layer)

In [53]:
max_1_pooling.shape

TensorShape([None, 3446, 23, 64])

In [54]:
# 2nd Convolutional Layer
conv_2_layer = Conv2D(filters=256, kernel_size=(7,7), activation='relu', 
                      strides=(1,1), padding='valid')(max_1_pooling)

In [55]:
conv_2_layer.shape

TensorShape([None, 3440, 17, 256])

In [56]:
# Max Pooling
max_2_pooling = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(conv_2_layer)

In [57]:
max_2_pooling.shape

TensorShape([None, 1720, 8, 256])

In [58]:
# 3rd Convolutional Layer
conv_3_layer =  Conv2D(filters=512, kernel_size=(3,3), activation='relu', 
                      strides=(1,1), padding='valid')(max_2_pooling)

In [59]:
conv_3_layer.shape

TensorShape([None, 1718, 6, 512])

In [60]:
# 4th Convolutional Layer
conv_4_layer = Conv2D(filters=512, kernel_size=(3,3), activation='relu', 
                      strides=(1,1), padding='valid')(conv_3_layer)

In [61]:
conv_4_layer.shape

TensorShape([None, 1716, 4, 512])

In [62]:
# 5th Convolutional Layer
conv_5_layer = Conv2D(filters=256, kernel_size=(3,3), activation='relu', 
                      strides=(1,1), padding='valid')(conv_4_layer)

In [63]:
conv_5_layer.shape

TensorShape([None, 1714, 2, 256])

In [64]:
# Max Pooling
max_5_pooling = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(conv_5_layer)

In [65]:
max_5_pooling.shape

TensorShape([None, 857, 1, 256])

In [66]:
# Fully Connected layer
fully_connected = Flatten()(max_5_pooling)

In [67]:
fully_connected.shape

TensorShape([None, 219392])

In [70]:
# 1st Fully Connected Layer
deep_dense_1_layer = Dense(1024, activation='relu')(fully_connected)
deep_dense_1_layer = Dropout(0.4)(deep_dense_1_layer) # <-------- [HyperParameter]

In [71]:
deep_dense_1_layer.shape

TensorShape([None, 1024])

In [72]:
# 2nd Fully Connected Layer
deep_dense_2_layer = Dense(1024, activation='relu')(deep_dense_1_layer)
deep_dense_2_layer = Dropout(0.4)(deep_dense_2_layer) # <-------- [HyperParameter]

In [73]:
deep_dense_2_layer.shape

TensorShape([None, 1024])

In [74]:
# 3rd Fully Connected Layer
deep_dense_3_layer = Dense(128, activation='relu')(deep_dense_2_layer)
deep_dense_3_layer = Dropout(0.4)(deep_dense_3_layer) # <-------- [HyperParameter]

In [75]:
deep_dense_3_layer.shape

TensorShape([None, 128])

In [76]:
predictions = Dense(K, activation='softmax')(deep_dense_3_layer)

In [77]:
#Criticality Model
criticality_network = Model(inputs=[gram_input],outputs=[predictions]) 

In [78]:
print(criticality_network.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 13789, 100, 1)]   0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6892, 47, 64)      3200      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 3446, 23, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3440, 17, 256)     803072    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 1720, 8, 256)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 1718, 6, 512)      1180160   
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 1716, 4, 512)      235980

In [79]:
#Seting up the Model
criticality_network.compile(optimizer='adam',loss='binary_crossentropy',
                                  metrics=['accuracy'])

In [80]:
#Data set organization
from tempfile import mkdtemp
import os.path as path

In [81]:
#Memoization 
file_corpora_train_x = path.join(mkdtemp(), 'alex-res_temp_corpora_train_x.dat') #Update per experiment
file_corpora_test_x = path.join(mkdtemp(), 'alex-res_temp_corpora_test_x.dat')

In [82]:
#Shaping
shape_train_x = (len(corpora_train),max_len_sentences,embeddigs_cols,1)
shape_test_x = (len(corpora_test),max_len_sentences,embeddigs_cols,1)

In [83]:
#Data sets
corpora_train_x = np.memmap(
        filename = file_corpora_train_x, 
        dtype='float32', 
        mode='w+', 
        shape = shape_train_x)

In [84]:
corpora_test_x = np.memmap( #Test Corpora (for future evaluation)
        filename = file_corpora_test_x, 
        dtype='float32', 
        mode='w+', 
        shape = shape_test_x)

In [85]:
target_train_y = np.array(target_train) #Train Target
target_test_y = np.array(target_test) #Test Target (for future evaluation)

In [86]:
corpora_train_x.shape

(9308, 13789, 100, 1)

In [87]:
target_train_y.shape

(9308, 2)

In [88]:
corpora_test_x.shape

(1030, 13789, 100, 1)

In [89]:
target_test_y.shape

(1030, 2)

In [90]:
#Reshaping Train Inputs
for doc in range(len(corpora_train)):
    #print(corpora_train[doc].shape[1])
    for words_rows in range(corpora_train[doc].shape[0]):
        embed_flatten = np.array(corpora_train[doc][words_rows]).flatten() #<--- Capture doc and word
        for embedding_cols in range(embed_flatten.shape[0]):
            corpora_train_x[doc,words_rows,embedding_cols,0] = embed_flatten[embedding_cols]

In [91]:
#Reshaping Test Inputs (for future evaluation)
for doc in range(len(corpora_test)):
    for words_rows in range(corpora_test[doc].shape[0]):
        embed_flatten = np.array(corpora_test[doc][words_rows]).flatten() #<--- Capture doc and word
        for embedding_cols in range(embed_flatten.shape[0]):
            corpora_test_x[doc,words_rows,embedding_cols,0] = embed_flatten[embedding_cols]

In [92]:
#CheckPoints
#csv_logger = CSVLogger(system+'_training.log')
filepath = "alex-res/embeds100-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)
callbacks_list = [checkpoint]

In [ ]:
#Model Fitting
history = criticality_network.fit(
            x = corpora_train_x, 
            y = target_train_y,
            #batch_size=64,
            epochs=15, #5 <------ Hyperparameter
            validation_split = 0.2,
            callbacks=callbacks_list
)

Train on 7446 samples, validate on 1862 samples
Epoch 1/15
3744/7446 [==============>...............] - ETA: 1:19:36 - loss: 0.7026 - accuracy: 0.5433

In [ ]:
#Saving Training History
df_history = pd.DataFrame.from_dict(history.history)
df_history.to_csv('C-res/history_training.csv', encoding='utf-8',index=False)

In [ ]:
df_history.head()

In [ ]:
#Saving Test Data
np.save('C-res/corpora_test_x.npy',corpora_test_x)
np.save('C-res/target_test_y.npy',target_test_y)

In [ ]:
#Evaluation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs2 = range(len(acc))
 
plt.plot(epochs2, acc, 'b', label='Training')
plt.plot(epochs2, val_acc, 'r', label='Validation')
plt.title('Training and validation accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend()
 
plt.figure()
 
plt.plot(epochs2, loss, 'b', label='Training')
plt.plot(epochs2, val_loss, 'r', label='Validation')
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
 
plt.show()